In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
print(tf.config.list_physical_devices('GPU'))
!nvidia-smi

In [ ]:
from keras.layers import Resizing, Rescaling
from sklearn.preprocessing import LabelBinarizer

enc = LabelBinarizer()

y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

print(x_train.shape)
print(x_test.shape)

x_train, y_train = np.concatenate((x_train, x_train), axis=0), np.concatenate((y_train, y_train), axis=0)
x_train, y_train = np.concatenate((x_train, x_train), axis=0), np.concatenate((y_train, y_train), axis=0)

print(x_train.shape)
print(x_test.shape)


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, RandomFlip, RandomRotation, RandomZoom
from keras.metrics import TopKCategoricalAccuracy

resize_rescale = tf.keras.Sequential([
    Input(shape=(32, 32, 3)),
    Resizing(299, 299),
    Rescaling(1./255, offset=-1),
    RandomFlip('horizontal_and_vertical'),
    RandomRotation(0.2),
    RandomZoom(0.2)
])
# create the base pre-trained model
base_model = InceptionV3(input_tensor=resize_rescale.output, weights='imagenet', include_top=False)


# add own top to pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(100, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first train only the top layers
# freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])




In [ ]:

from keras.metrics import TopKCategoricalAccuracy

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 255 layers and unfreeze the rest:
for layer in model.layers[:255]:
   layer.trainable = False
for layer in model.layers[255:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])


In [ ]:
!pip install pyNVML
import pynvml as nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
total_gpu_memory = info.total
# Weights are float32 -> 4 bytes
model_size = model.count_params() * 4
# Rescaling layer returns float32
# 299*299 pixels, RGB colors
image_size = 4 * 299 * 299 * 3
MAX_BATCH_SIZE = int(np.floor(total_gpu_memory / (model_size + image_size)))
MAX_BATCH_SIZE_LOG2 = int(2 ** np.floor(np.log2(MAX_BATCH_SIZE)))
print(MAX_BATCH_SIZE)
print(MAX_BATCH_SIZE_LOG2)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, TerminateOnNaN
import datetime

logdir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model.fit(x_train, y_train,
          epochs=200,
          batch_size=MAX_BATCH_SIZE_LOG2,
          validation_batch_size=MAX_BATCH_SIZE_LOG2,
          validation_split=0.2,
          verbose=1,
          callbacks=[
              TensorBoard(
                  log_dir=logdir,
                  histogram_freq=1
              ),
              EarlyStopping(
                  monitor='val_top_1_accuracy',
                  patience=5,
              ),
              ModelCheckpoint(
                  logdir+'/checkpoint',
                  monitor='val_top_1_accuracy',
                  save_best_only=True,
              )
          ],
)


In [ ]:
# train 2 more blocks
for layer in model.layers[:199]:
   layer.trainable = False
for layer in model.layers[199:]:
   layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])

logdir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

model.fit(x_train, y_train,
          epochs=200,
          batch_size=MAX_BATCH_SIZE_LOG2,
          validation_batch_size=MAX_BATCH_SIZE_LOG2,
          validation_split=0.2,
          verbose=1,
          callbacks=[
              TensorBoard(
                  log_dir=logdir,
                  histogram_freq=1
              ),
              EarlyStopping(
                  monitor='val_top_1_accuracy',
                  patience=5,
              ),
              ModelCheckpoint(
                  logdir+'/checkpoint',
                  monitor='val_top_1_accuracy',
                  save_best_only=True,
              )
          ],
)

In [ ]:
%load_external tensorboard

In [ ]:
%tensorboard --logdir ./logs/fit